In [8]:
import pandas as pd
import os
from typing import Union, List, Dict, Tuple, Optional
from collections import Counter

In [9]:
def read_excel_sheets(file_path: str,
                     sheets: Union[str, List[str]] = None,
                     na_values: List[str] = None,
                     dtype: Dict[str, str] = None) -> Union[pd.DataFrame, Dict[str, pd.DataFrame]]:
   '''
   Reads Excel sheets into Pandas DataFrame(s).


   Args:
       file_path (str): Path to the Excel file.
       sheets (Union[str, List[str]], optional): Sheet name or list of sheet names to load. If None, loads the first sheet.
       na_values (List[str], optional): Additional strings to recognize as NA/NaN.
       dtype (Dict[str, str], optional): Column data types.


   Returns:
       Union[pd.DataFrame, Dict[str, pd.DataFrame]]: DataFrame for one sheet, or dict of DataFrames if multiple sheets.
   '''
   if not os.path.isfile(file_path):
       raise FileNotFoundError(f'File not found: {file_path}')


   print(f'Reading file: {file_path}')


   df = pd.read_excel(
       file_path,
       sheet_name=sheets,
       na_values=na_values,
       dtype=dtype
   )


   return df

In [10]:
def explore_df(df: pd.DataFrame) -> None:
   '''
   Quickly explore a DataFrame.
   '''
   print('Shape:', df.shape)
   print('\nColumns:', df.columns.tolist())
   print('\nFirst 5 rows:')
   print(df.head())
   print('\n10 Random rows:')
   print(df.sample(10))
   print('\nInfo:')
   print(df.info())
   print('\nDescribe:')
   print(df.describe(include='all'))
   print('\nMissing values per column:')
   print(df.isnull().sum())

In [11]:
def handle_ratios(value) -> Tuple[Optional[float], Optional[float], Optional[float], Optional[float]]:
    '''
    Parses a simple ratio string like '2:1' and returns:
    - low: the smaller of the two numbers
    - high: the larger of the two numbers
    - midpoint: arithmetic mean of the two values
    - ratio: first number divided by the second

    Returns (None, None, None, None) for invalid input (e.g., wrong format, division by zero).
    '''
    try:
        # Convert the input to string, strip whitespace, and split by ':'
        parts = list(map(float, str(value).strip().split(':')))
        
        # Ensure the ratio has exactly two numeric parts and avoid division by zero
        if len(parts) != 2 or parts[1] == 0:
            return None, None, None, None
        
        # Get the smaller and larger of the two values
        low = min(parts)
        high = max(parts)

        # Calculate the midpoint (arithmetic mean of the two numbers)
        midpoint = sum(parts) / 2

        # Compute the directional ratio using original order (a / b)
        ratio = parts[0] / parts[1]

        # Return all four computed values
        return low, high, midpoint, ratio

    # Catch any unexpected errors (e.g., non-numeric input)
    except Exception:
        return None, None, None, None

In [12]:
def extract_list_from_string(raw_string):
    '''
    Converts a comma-separated string to a list of stripped strings.
    Handles missing or non-string inputs safely.
    '''
    if not isinstance(raw_string, str):
        return []
    return [item.strip() for item in raw_string.split(',') if item.strip()]

Currently, beer_styles isn't being entirely extracted correctly by ScrapStorm.

In [13]:
# === Load and Combine Excel Sheets ===
sheets_to_load = [
    'beer_maverick_rows_1_100'
]

# Load specified sheets from Excel file.

hop_df_multi = read_excel_sheets('beer_maverick_data.xlsx', sheets=sheets_to_load)

# Display basic info about each loaded sheet.
for sheet_name, df in hop_df_multi.items():
    print(f'Loaded {sheet_name}: {df.shape}')

# Concatenate all sheets into a single df.
hop_df = pd.concat(hop_df_multi.values(), ignore_index=True)

# Initial exploration before any parsing.
print('Before parsing columns:')
explore_df(hop_df)

Reading file: beer_maverick_data.xlsx
Loaded beer_maverick_rows_1_100: (100, 14)
Before parsing columns:
Shape: (100, 14)

Columns: ['name', 'purpose', 'alpha_acid_avg', 'beta_acid_avg', 'alpha_beta_ratio', 'hsi', 'cohumulone_%_of_alpha', 'total_oils_avg', 'myrcene', 'humulene', 'caryophyllene', 'farnesene', 'tags', 'beer_styles']

First 5 rows:
      name purpose  alpha_acid_avg  beta_acid_avg alpha_beta_ratio    hsi  \
0      NaN     NaN             NaN            NaN              NaN    NaN   
1    Astra    Dual             8.5            5.0              2:1    NaN   
2  Eclipse    Dual            17.4            7.5              2:1    NaN   
3     Ella    Dual            16.3            5.9              3:1    NaN   
4   Enigma   Aroma            16.5            5.8              3:1  0.295   

  cohumulone_%_of_alpha  total_oils_avg  myrcene  humulene  caryophyllene  \
0                   NaN             NaN      NaN       NaN            NaN   
1               27% avg            

In [14]:
# === Clean and Parse Data ===
# Replace 'ratio' with the midpoint of the two numbers in the ratio string.
# For example, '2:1' becomes 1.5 = (2 + 1) / 2
hop_df['alpha_beta_ratio'] = hop_df['alpha_beta_ratio'].apply(lambda x: handle_ratios(x)[2])

# Loop through the columns 'beer_styles' & 'tags', apply the extract_list_from_string function.
# This converts each comma-separated string (e.g., 'IPA,Lager') into a list (e.g., ['IPA', 'Lager']).
# The original string values in these columns will be overwritten with their corresponding lists.
for col in ['beer_styles', 'tags']:
    hop_df[col] = hop_df[col].apply(extract_list_from_string)

    # Count and print all unique items in the column
    all_items = [item for sublist in hop_df[col] for item in sublist]
    counts = Counter(all_items)

    print(f'\nUnique values and counts for column: {col}')
    for item, count in counts.most_common():
        print(f'  {item}: {count}')

# Final exploration after parsing.
print('After parsing columns:')
explore_df(hop_df)

# Write out processed df to parquet file (preserves list structure, compressed).
parquet_path = 'processed_hop_data.parquet'
hop_df.to_parquet(parquet_path, index=False)
print(f'Data exported to: {parquet_path} (Parquet format)')

# Write out processed df to CSV.
csv_path = 'processed_hop_data.csv'
hop_df.to_csv(csv_path, index=False)
print(f'Data exported to: {csv_path} (CSV format — lists saved as strings)')


Unique values and counts for column: beer_styles
  Lager: 37
  IPA: 33
  Pale Ale: 29
  Pilsner: 27
  Belgian Ale: 17
  Saison: 14
  Ale: 9
  Wheat: 9
  Stout: 7
  Kolsch: 6
  NEIPA: 5
  Bock: 5
  Porter: 5
  German Ale: 5
  Altbier: 5
  Wheat Beer: 4
  Helles: 4
  American Ale: 3
  Bitter: 3
  ESB: 3
  Wheats: 3
  Golden Ale: 3
  English Ale: 3
  German Lager: 3
  Hefeweizen: 3
  Fruit Beers: 2
  American Pale Ale: 2
  Amber Ale: 2
  Red Ale: 2
  Lagers: 2
  Belgian Ales: 2
  French Ale: 2
  Blonde Ale: 2
  Dunkel: 2
  Sours: 2
  Ales: 2
  Wheat Ale: 1
  Barley Wine: 1
  Wild Ale: 1
  British Ale: 1
  Australian Ale: 1
  Australian Lager: 1
  Imperial Pale Ale: 1
  California Common: 1
  Whitbier: 1
  Pale Ales: 1
  Trappist: 1
  Weizenbier: 1
  Amber: 1
  Bière de Garde: 1
  Black IPA: 1
  Blonde: 1
  Farmhouse Ale: 1
  Bière de Gardes: 1
  Alt: 1
  English IPA: 1
  Dark Ale: 1
  Bretts: 1
  American Lager: 1
  Cask Ale: 1
  Strong Ale: 1
  Weizenbock: 1
  Marzen: 1
  Specialty Ale:

Very high missing values: 'hsi'
High missing values: 'myrcene', 'humulene', 'caryophyllene', 'farnesene'